In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np
import pandas as pd
import multiprocessing
import xarray as xr
import rioxarray
import os
import subprocess



# Preprocessing

In [3]:
total_area_path = '/shared/data/ghsl_data/2020/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'
nres_area_path = '/shared/data/ghsl_data/2020/GHS_BUILT_S_NRES_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_NRES_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'
total_floor_area_path = '/shared/data/building-emissions/data/ghsl/building_floor_area_2020.parquet'

rds = rioxarray.open_rasterio(total_area_path)
rds.name = "building_footprint_area"
# ghs = rds.squeeze().to_dataframe().reset_index()

# ghs_floor = pd.read_parquet(total_floor_area_path)

In [108]:
ghs_floor = pd.read_parquet(total_floor_area_path)
nres = pd.read_parquet('/shared/data/building-emissions/data/ghsl/nres_building_floor_area_2020.parquet')

In [111]:
ghs_floor

,lat,lon,building_floor_area_total,building_floor_area_nres,building_floor_area_res
0,8999500.0,-18040500.0,4.294967e+09,4.294967e+09,0.0
1,8999500.0,-18039500.0,4.294967e+09,4.294967e+09,0.0
2,8999500.0,-18038500.0,4.294967e+09,4.294967e+09,0.0
3,8999500.0,-18037500.0,4.294967e+09,4.294967e+09,0.0
4,8999500.0,-18036500.0,4.294967e+09,4.294967e+09,0.0
...,...,...,...,...,...
649475995,-8999500.0,18036500.0,4.294967e+09,4.294967e+09,0.0
649475996,-8999500.0,18037500.0,4.294967e+09,4.294967e+09,0.0
649475997,-8999500.0,18038500.0,4.294967e+09,4.294967e+09,0.0
649475998,-8999500.0,18039500.0,4.294967e+09,4.294967e+09,0.0


In [110]:
ghs_floor['building_floor_area_nres'] = nres['nres_building_floor_area']
ghs_floor['building_floor_area_res'] = ghs_floor['building_floor_area'] - ghs_floor['building_floor_area_nres']
ghs_floor.rename(columns={'building_floor_area': 'building_floor_area_total'}, inplace=True)


In [86]:
footprint_area_tif_t_path =\
     '/shared/data/ghsl_data/2020/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'
footprint_area_tif_nres_path =\
       '/shared/data/ghsl_data/2020/GHS_BUILT_S_NRES_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_NRES_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'


In [11]:
def grid_to_gdf(grid_path_total, grid_path_nres):#, gdf_final_dir):
    """
    Convert grid to geodataframe with res/nres.total footprint_area before chunking
    Input:
        grid_path_total: path to total footprint_area grid dataframe
        grid_path_nres: path to non-residential footprint_area grid dataframe
        gdf_final_dir: directory to save the final gdf
    Output:
        grid_total: total footprint_area grid dataframe
        gdf_final: final gdf with res/nres/total footprint_area
    """
    grid_total = pd.read_parquet(grid_path_total)
    gdf_total = grid_total.transpose().unstack().reset_index()
    gdf_total.columns = ["lat", "lon", "footprint_area_total"]

    grid_nres = pd.read_parquet(grid_path_nres)
    gdf_nres = grid_nres.transpose().unstack().reset_index()
    gdf_nres.columns = ["lat", "lon", "footprint_area_nres"]
    # merge gdf_total and gdf_nres
    gdf_total["footprint_area_nres"] = gdf_nres["footprint_area_nres"]
    gdf_total["footprint_area_res"] = gdf_total["footprint_area_total"] - gdf_total["footprint_area_nres"]
    # gdf_total.to_parquet(
    #     os.path.join(gdf_final_dir, "gdf_before_chunk.parquet"), index=False
    # )
    gdf_final = gdf_total.copy()

    return grid_total, gdf_final


In [87]:
total_footprint_area_path = '/shared/data/ghsl_data/2020/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'
nres_footprint_area_path = '/shared/data/ghsl_data/2020/GHS_BUILT_S_NRES_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_NRES_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'

rds = rioxarray.open_rasterio(total_footprint_area_path)
rds.name = "building_footprint_area_total"
total_footprint = rds.squeeze().to_dataframe().reset_index()

rds = rioxarray.open_rasterio(nres_footprint_area_path)
rds.name = "building_footprint_area_nres"
nres_footprint = rds.squeeze().to_dataframe().reset_index()


In [97]:
ghs_floor

,lat,lon,building_floor_area_total,building_floor_area_nres,building_footprint_area_res,building_footprint_area_total,building_footprint_area_nres
0,8999500.0,-18040500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
1,8999500.0,-18039500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
2,8999500.0,-18038500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
3,8999500.0,-18037500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
4,8999500.0,-18036500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
...,...,...,...,...,...,...,...
649475995,-8999500.0,18036500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
649475996,-8999500.0,18037500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
649475997,-8999500.0,18038500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295
649475998,-8999500.0,18039500.0,4.294967e+09,4.294967e+09,0,4294967295,4294967295


In [112]:
ghs_floor['building_footprint_area_total'] = total_footprint['building_footprint_area_total']
ghs_floor['building_footprint_area_nres'] = nres_footprint['building_footprint_area_nres']
ghs_floor['building_footprint_area_res'] = ghs_floor['building_footprint_area_total'] - ghs_floor['building_footprint_area_nres']

In [113]:
ghs_floor

,lat,lon,building_floor_area_total,building_floor_area_nres,building_floor_area_res,building_footprint_area_total,building_footprint_area_nres,building_footprint_area_res
0,8999500.0,-18040500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
1,8999500.0,-18039500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
2,8999500.0,-18038500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
3,8999500.0,-18037500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
4,8999500.0,-18036500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
...,...,...,...,...,...,...,...,...
649475995,-8999500.0,18036500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
649475996,-8999500.0,18037500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
649475997,-8999500.0,18038500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0
649475998,-8999500.0,18039500.0,4.294967e+09,4.294967e+09,0.0,4294967295,4294967295,0


In [114]:
ghs_floor.to_parquet('/shared/data/building-emissions/data/ghsl/building_floor_footprint_area_ALL_2020.parquet')

# Chunking

In [2]:
################## START: multiprocessing of chunking grid and gdf ##################
def chunk_grid_multiprocess(
    i,
    chunk_size,
    k,
    # dest_dir_chunked_grid,
    dest_dir_chunked_gdf,
    dest_dir_chunked_full_gdf,
    n_lon_values
):
    """
    Child function to chunk grid and gdf into smaller dataframes, save to destination directory
    Input:
        i: starting row number
        chunk_size: number of rows in each chunk
        k: chunk number
        dest_dir_chunked_grid: directory to save grid chunks
        dest_dir_chunked_gdf: directory to save gdf chunks
        dest_dir_chunked_full_gdf: directory to save full gdf chunks
    Output:
        None
    """
    # grid_chunk = grid_total.iloc[i : i + chunk_size]
    # grid_chunk.to_parquet(
    #     os.path.join(dest_dir_chunked_grid, f"grid_chunk_{k}.parquet"), index=True
    # )

    gdf_chunk = gdf_final.iloc[i * n_lon_values : (i + chunk_size) * n_lon_values]
    # keep only lat and lon columns
    full_gdf_chunk = gdf_chunk[["lat", "lon"]]
    full_gdf_chunk.to_parquet(
        os.path.join(dest_dir_chunked_full_gdf, f"gdf_full_chunk_{k}.parquet"),
        index=False,
    )
    gdf_chunk_subset = gdf_chunk[
        (gdf_chunk["building_footprint_area_total"] > 0) & (gdf_chunk["building_footprint_area_total"] < 4294967295)
    ]
    gdf_chunk_subset.to_parquet(
        os.path.join(dest_dir_chunked_gdf, f"gdf_chunk_{k}.parquet"), index=False
    )

    pass


def child_initialize_grid(_lat_values, _gdf_final):
    """
    Help function to make grid_total and gdf_final global variables
    Input:
        _lat_values grid dataframe
        _gdf_final: gdf with total, nres and res area
    """
    global lat_values, gdf_final
    lat_values = _lat_values
    gdf_final = _gdf_final
    pass


def chunk_grid_gdf(
    grid_path,
    area_path,
    n_chunks,
    dest_dir_chunked_gdf,
    dest_dir_chunked_full_gdf,
):
    """
    Multiprocessing chunking grid and gdf into smaller dataframes, save to destination directory
    Input:
        grid_path: path to grid
        area_path: path to area
        n_chunks: number of chunks
        dest_dir_chunked_grid: directory to save grid chunks
        dest_dir_chunked_gdf: directory to save gdf chunks
        dest_dir_chunked_full_gdf: directory to save full gdf chunks
    Output:
        None
    """
    rds = rioxarray.open_rasterio(grid_path)
    lat_values = rds.y.values
    n_lon_values = rds.x.values.size
    
    gdf_final = pd.read_parquet(area_path)

    # use multiprocessing to speed up
    num_cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(
        processes=num_cores,
        initializer=child_initialize_grid,
        initargs=(lat_values, gdf_final),
    )
    chunk_size = len(lat_values) // n_chunks

    k = 0
    for i in range(0, len(lat_values), chunk_size):
        pool.apply_async(
            chunk_grid_multiprocess,
            args=(
                i,
                chunk_size,
                k,
                dest_dir_chunked_gdf,
                dest_dir_chunked_full_gdf,
                n_lon_values
            ),
        )
        k += 1
    pool.close()
    pool.join()

    pass


In [3]:
# path to all area
area_path = '/shared/data/building-emissions/data/ghsl/building_floor_footprint_area_ALL_2020.parquet'
# path to grid
grid_path = '/shared/data/ghsl_data/2020/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0/GHS_BUILT_S_E2020_GLOBE_R2023A_54009_1000_V1_0.tif'

# destination directory to save gdf chunks
dest_dir_chunked_gdf = '/shared/data/building-emissions/data/estimates2020/chunked_gdf'
# destination directory to save full gdf chunks
dest_dir_chunked_full_gdf = '/shared/data/building-emissions/data/estimates2020/chunked_full_gdf'

# number of chunks
n_chunks = 200

chunk_grid_gdf(
    grid_path,
    area_path,
    n_chunks,
    dest_dir_chunked_gdf,
    dest_dir_chunked_full_gdf,
)


# Preprocessing of EUI and EF

In [34]:
eui_ef_path = f'/shared/data/building-emissions/data/estimates2020/eui_ef_chunks/'
merged_eui_ef_0 = gpd.read_parquet(os.path.join(eui_ef_path, 'eui_ef_0.parquet'))
merged_eui_ef_1 = gpd.read_parquet(os.path.join(eui_ef_path, 'eui_ef_1.parquet'))
merged_eui_ef_2 = gpd.read_parquet(os.path.join(eui_ef_path, 'eui_ef_2.parquet'))
merged_eui_ef = gpd.GeoDataFrame(pd.concat([merged_eui_ef_0, merged_eui_ef_1, merged_eui_ef_2]))


In [40]:

def merge_eui_ef(eui_chunk_dir, ef_path, dest_dir):
    '''
    Merge eui and ef dataframes, save to destination directory
    '''
    eui_0 = gpd.read_parquet(os.path.join(eui_chunk_dir, 'eui_0.parquet'))
    eui_1 = gpd.read_parquet(os.path.join(eui_chunk_dir, 'eui_1.parquet'))
    eui_2 = gpd.read_parquet(os.path.join(eui_chunk_dir, 'eui_2.parquet'))
    eui = pd.concat([eui_0, eui_1, eui_2])

    eui.to_crs('ESRI:54009', inplace=True)
    eui.loc[:, 'Non-residential EUI (mj)'] = eui.loc[:, 'Non-residential EUI (kwh)'] * 3.6
    eui.loc[:, 'Residential EUI (mj)'] = eui.loc[:, 'Residential EUI (kwh)'] * 3.6
    ef = gpd.read_parquet(ef_path)
    ef.to_crs('ESRI:54009', inplace=True)
    merged_eui_ef = eui.merge(ef, how='left', on='gid0', suffixes=('_eui', '_ef'))    
    merged_eui_ef_gdf = gpd.GeoDataFrame(merged_eui_ef, geometry='geometry_eui', crs = 'ESRI:54009')
    merged_eui_ef_gdf.drop(columns=['geometry_ef'], inplace=True)
    # chunk merged eui_ef_gdf to 3 chunks
    n = len(merged_eui_ef_gdf)//3
    merged_eui_ef_gdf_0 = merged_eui_ef_gdf.iloc[:n]
    merged_eui_ef_gdf_1 = merged_eui_ef_gdf.iloc[n:2*n]
    merged_eui_ef_gdf_2 = merged_eui_ef_gdf.iloc[2*n:]
    merged_eui_ef_gdf_0.to_parquet(os.path.join(dest_dir, 'eui_ef_0.parquet'))
    merged_eui_ef_gdf_1.to_parquet(os.path.join(dest_dir, 'eui_ef_1.parquet'))
    merged_eui_ef_gdf_2.to_parquet(os.path.join(dest_dir, 'eui_ef_2.parquet'))    
    return merged_eui_ef_gdf



In [41]:
eui_chunked_dir = '/shared/data/building-emissions/data/energyuseintensities/eui_chunks/'
ef_path = '/shared/data/building-emissions/data/emissionsfactors/global_emissionsfactors_nobiofuel.parquet'
dest_dir = '/shared/data/building-emissions/data/estimates2020/eui_ef_chunks/'
merge_eui_ef(eui_chunked_dir, ef_path, dest_dir)

,level,gid0,gid1,gid2,name,orig_fid,gadm_id,geometry_eui,Non-residential EUI (kwh),Residential EUI (kwh),Non-residential EUI (mj),Residential EUI (mj),Residential_Methane,Residential_CO2,Residential_N2O,Commercial_Methane,Commercial_CO2,Commercial_N2O
0,-1,None,None,None,Africa,NaN,None,"MULTIPOLYGON (((3398976.666 -2656607.978, 3394...",104.910096,71.739117,377.676346,258.260822,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,None,None,None,Asia,NaN,None,"MULTIPOLYGON (((-9433012.258 7688681.960, -943...",135.967905,94.194403,489.484457,339.099851,NaN,NaN,NaN,NaN,NaN,NaN
2,-1,None,None,None,Australia,NaN,None,"MULTIPOLYGON (((11295905.055 -6357557.661, 112...",174.410095,134.877978,627.876341,485.560721,NaN,NaN,NaN,NaN,NaN,NaN
3,-1,None,None,None,North America,NaN,None,"MULTIPOLYGON (((-8144037.095 912592.474, -8141...",201.837252,107.196552,726.614108,385.907588,NaN,NaN,NaN,NaN,NaN,NaN
4,-1,None,None,None,Oceania,NaN,None,"MULTIPOLYGON (((17551075.398 -2085950.976, 175...",136.826690,97.823870,492.576085,352.165930,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51134,2,GHA,GHA14_2,GHA14.10_2,None,19338.0,GHA14.10_2,"MULTIPOLYGON (((47810.092 900725.256, 47955.22...",106.566370,65.641164,383.638931,236.308190,0.320312,3708.750,0.046094,0.333157,3578.202,0.025627
51135,2,GHA,GHA14_2,GHA14.11_2,None,19339.0,GHA14.11_2,"MULTIPOLYGON (((86197.947 749952.461, 86255.28...",106.566370,65.641164,383.638931,236.308190,0.320312,3708.750,0.046094,0.333157,3578.202,0.025627
51136,2,GHA,GHA14_2,GHA14.12_2,None,19340.0,GHA14.12_2,"MULTIPOLYGON (((107766.808 762468.384, 107608....",106.566370,65.641164,383.638931,236.308190,0.320312,3708.750,0.046094,0.333157,3578.202,0.025627
51137,2,GHA,GHA14_2,GHA14.3_2,None,19331.0,GHA14.3_2,"MULTIPOLYGON (((74795.857 796640.412, 74727.24...",106.566370,65.641164,383.638931,236.308190,0.320312,3708.750,0.046094,0.333157,3578.202,0.025627


# calculation

In [9]:
def calculate_chunk(finest_gdf_chunk_eui_ef):
    # calculation * 18 columns
    # eui*area, total/res/nres * c02/methane/n20, total/res/nres * co2e * 20/100
    # foot print
    finest_gdf_chunk_eui_ef['footprint_nres_eui*area'] = finest_gdf_chunk_eui_ef['building_footprint_area_nres'] * finest_gdf_chunk_eui_ef['Non-residential EUI (mj)']
    finest_gdf_chunk_eui_ef['footprint_res_eui*area'] = finest_gdf_chunk_eui_ef['building_footprint_area_res'] * finest_gdf_chunk_eui_ef['Residential EUI (mj)']
    
    finest_gdf_chunk_eui_ef['footprint_nres_co2_emission'] = finest_gdf_chunk_eui_ef['footprint_nres_eui*area'] * finest_gdf_chunk_eui_ef['Commercial_CO2']
    finest_gdf_chunk_eui_ef['footprint_nres_methane_emission'] = finest_gdf_chunk_eui_ef['footprint_nres_eui*area'] * finest_gdf_chunk_eui_ef['Commercial_Methane']
    finest_gdf_chunk_eui_ef['footprint_nres_n20_emission'] = finest_gdf_chunk_eui_ef['footprint_nres_eui*area'] * finest_gdf_chunk_eui_ef['Commercial_N2O']
    finest_gdf_chunk_eui_ef['footprint_res_co2_emission'] = finest_gdf_chunk_eui_ef['footprint_res_eui*area'] * finest_gdf_chunk_eui_ef['Residential_CO2']
    finest_gdf_chunk_eui_ef['footprint_res_methane_emission'] = finest_gdf_chunk_eui_ef['footprint_res_eui*area'] * finest_gdf_chunk_eui_ef['Residential_Methane']
    finest_gdf_chunk_eui_ef['footprint_res_n20_emission'] = finest_gdf_chunk_eui_ef['footprint_res_eui*area'] * finest_gdf_chunk_eui_ef['Residential_N2O']

    # floor
    finest_gdf_chunk_eui_ef['floor_nres_eui*area'] = finest_gdf_chunk_eui_ef['building_floor_area_nres'] * finest_gdf_chunk_eui_ef['Non-residential EUI (mj)']
    finest_gdf_chunk_eui_ef['floor_res_eui*area'] = finest_gdf_chunk_eui_ef['building_floor_area_res'] * finest_gdf_chunk_eui_ef['Residential EUI (mj)']
    
    finest_gdf_chunk_eui_ef['floor_nres_co2_emission'] = finest_gdf_chunk_eui_ef['floor_nres_eui*area'] * finest_gdf_chunk_eui_ef['Commercial_CO2']
    finest_gdf_chunk_eui_ef['floor_nres_methane_emission'] = finest_gdf_chunk_eui_ef['floor_nres_eui*area'] * finest_gdf_chunk_eui_ef['Commercial_Methane']
    finest_gdf_chunk_eui_ef['floor_nres_n20_emission'] = finest_gdf_chunk_eui_ef['floor_nres_eui*area'] * finest_gdf_chunk_eui_ef['Commercial_N2O']
    finest_gdf_chunk_eui_ef['floor_res_co2_emission'] = finest_gdf_chunk_eui_ef['floor_res_eui*area'] * finest_gdf_chunk_eui_ef['Residential_CO2']
    finest_gdf_chunk_eui_ef['floor_res_methane_emission'] = finest_gdf_chunk_eui_ef['floor_res_eui*area'] * finest_gdf_chunk_eui_ef['Residential_Methane']
    finest_gdf_chunk_eui_ef['floor_res_n20_emission'] = finest_gdf_chunk_eui_ef['floor_res_eui*area'] * finest_gdf_chunk_eui_ef['Residential_N2O']
    
    return finest_gdf_chunk_eui_ef

def finest_eui(merged_gdf_chunk_eui_ef):
    '''
    Find the finest-level of fidelity for each grid cell
    '''
    merged_gdf_chunk_eui_ef.reset_index(inplace=True)
    finest_gdf_chunk_eui_ef = merged_gdf_chunk_eui_ef.sort_values('level', ascending=False).drop_duplicates('index').sort_index().reset_index(drop = True)
    
    return finest_gdf_chunk_eui_ef

def layover_eui_ef(gdf_subchunk, results):
    """Match grid chunks to eui_ef datframe and do calculation to get emission for each grid chunk, return a dataframe with emission of each GHG"""
    merged_gdf_chunk_eui_ef = gpd.sjoin(gdf_subchunk, merged_eui_ef, how='left', predicate='intersects')
    # find finest level of fidelity
    finest_gdf_chunk_eui_ef = finest_eui(merged_gdf_chunk_eui_ef)
    # drop geometry column to save memory
    finest_gdf_chunk_eui_ef.drop(columns = ['geometry'], inplace = True)

    # calculation * 2(res/nres)* 4 columns(eui*area/co2/methane/n20)
    calculated_chunk = calculate_chunk(finest_gdf_chunk_eui_ef)
    # append to results
    results.append(calculated_chunk)
    return results
    # pass

def child_initialize(_merged_eui_ef):
    """function for multiprocessing to pass eui_ef to each child process"""
    global merged_eui_ef
    merged_eui_ef = _merged_eui_ef
    pass

def chunk_data(data, n_chunks):
    """Function to chunk data for multiprocessing"""
    chunk_size = len(data)//n_chunks + 1
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]

def iterate_chunks(gdf_chunk, merged_eui_ef, n_chunks = 64):
    """Match gdf chunks to eui and ef"""

    gdf_chunk_chunks = list(chunk_data(gdf_chunk, n_chunks))
    num_processes = multiprocessing.cpu_count()  # Use the number of available CPU cores
    pool = multiprocessing.Pool(processes=num_processes, initializer=child_initialize, initargs=(merged_eui_ef,))
    manager = multiprocessing.Manager()
    results = manager.list()
    for gdf_subchunks in gdf_chunk_chunks:
        pool.apply_async(layover_eui_ef, args=(gdf_subchunks, results))
    pool.close()
    pool.join()
    
    if len(results) == 0:
        return None
    calculated_concated_chunks = pd.concat(results)

    # sort the concat result by latitude then longitude
    # calculated_concated_chunks = calculated_concated_chunks.sort_values(['lat', 'lon']).reset_index(drop = True)    
    return calculated_concated_chunks

def cal_by_gdf_chunks(gdf_chunks_dir, eui_ef_dir, calculated_gdf_chunk_dir, n_chunks_within_chunk = 60, range_min = 0, range_max = 300):
    """"Calculate emissions based on grid area, eui and ef values"""
    eui_ef_0 = gpd.read_parquet(os.path.join(eui_ef_dir, 'eui_ef_0.parquet'))
    eui_ef_1 = gpd.read_parquet(os.path.join(eui_ef_dir, 'eui_ef_1.parquet'))
    eui_ef_2 = gpd.read_parquet(os.path.join(eui_ef_dir, 'eui_ef_2.parquet'))
    merged_eui_ef = gpd.GeoDataFrame(pd.concat([eui_ef_0, eui_ef_1, eui_ef_2]))
    print('eui_ef loaded')
    for i in range(range_min, range_max):
        if i% 50== 0:
            print(i)
        gdf_chunk = pd.read_parquet(f'{gdf_chunks_dir}/gdf_chunk_{i}.parquet')
        if len(gdf_chunk) == 0:
            continue
        gdf_chunk = gpd.GeoDataFrame(gdf_chunk, geometry=gpd.points_from_xy(gdf_chunk.lon, gdf_chunk.lat), crs="ESRI:54009")
        calculated_chunk = iterate_chunks(gdf_chunk, merged_eui_ef, n_chunks_within_chunk)
        if calculated_chunk is not None:
            calculated_chunk.to_parquet(f'{calculated_gdf_chunk_dir}/calculated_gdf_chunk_{i}.parquet')


In [14]:
gdf_chunks_dir = '/shared/data/building-emissions/data/estimates2020/chunked_gdf'
eui_ef_dir = '/shared/data/building-emissions/data/estimates2020/eui_ef_chunks'
calculated_gdf_chunk_dir = '/shared/data/building-emissions/data/estimates2020/calculated_gdf_chunks'
cal_by_gdf_chunks(gdf_chunks_dir, eui_ef_dir, calculated_gdf_chunk_dir, n_chunks_within_chunk = 64, range_min = 0, range_max = 200)

eui_ef loaded
0


# Map back to full gdf

In [ ]:
def process_chunk_to_grid(i, calculated_gdf_chunk_dir, dest_dir_chunked_full_gdf, calculated_full_gdf_dir):
    cal_gdf_chunk_path = f'{calculated_gdf_chunk_dir}/calculated_gdf_chunk_{i}.parquet'
    full_gdf_chunk_path = f'{dest_dir_chunked_full_gdf}/gdf_full_chunk_{i}.parquet'
    full_gdf = pd.read_parquet(full_gdf_chunk_path)
    # create column name list
    names = ['footprint_res_eui*area', 'footprint_res_co2_emission', 'footprint_res_methane_emission', 'footprint_res_n20_emission', \
             'footprint_nres_eui*area', 'footprint_nres_co2_emission', 'footprint_nres_methane_emission', 'footprint_nres_n20_emission', \
            'floor_res_eui*area', 'floor_res_co2_emission', 'floor_res_methane_emission', 'floor_res_n20_emission', \
            'floor_nres_eui*area', 'floor_nres_co2_emission', 'floor_nres_methane_emission', 'floor_nres_n20_emission']
    # add these column to the existing fullgdf with 0 value
    for name in names:
        full_gdf[name] = 0
    
    if os.path.exists(cal_gdf_chunk_path):
        cal_gdf_chunk = pd.read_parquet(cal_gdf_chunk_path)
        # merge calculated chunk to full gdf
        for j in range(len(cal_gdf_chunk)):
            lat, lon = cal_gdf_chunk.iloc[j]['lat'], cal_gdf_chunk.iloc[j]['lon']
            all_values = cal_gdf_chunk.iloc[j][['res_eui*area', 'res_co2_emission', 'res_methane_emission', 'res_n20_emission', \
                                            'nres_eui*area', 'nres_co2_emission', 'nres_methane_emission', 'nres_n20_emission']].values.astype(np.float64)
            all_values = np.round(all_values,2)
            # map value back to full_gdf
            full_gdf.loc[(full_gdf['lat'] == lat) & (full_gdf['lon'] == lon), names] = all_values

        del cal_gdf_chunk
    # co2 eq
    
    full_gdf['footprint_res_co2eq_20yr'] = full_gdf['footprint_res_co2_emission'] + \
                                            82.5*full_gdf['footprint_res_methane_emission'] + 273*full_gdf['footprint_res_n20_emission']
    full_gdf['footprint_nres_co2eq_20yr'] = full_gdf['footprint_nres_co2_emission'] + \
                                            82.5*full_gdf['footprint_nres_methane_emission'] + 273*full_gdf['footprint_nres_n20_emission']
    full_gdf['footprint_res_co2eq_100yr'] = full_gdf['footprint_res_co2_emission'] + \
                                            29.8*full_gdf['footprint_res_methane_emission'] + 273*full_gdf['footprint_res_n20_emission']
    full_gdf['footprint_nres_co2eq_100yr'] = full_gdf['footprint_nres_co2_emission'] + \
                                            29.8*full_gdf['footprint_nres_methane_emission'] + 273*full_gdf['footprint_nres_n20_emission']
    full_gdf['floor_res_co2eq_20yr'] = full_gdf['floor_res_co2_emission'] + \
                                            82.5*full_gdf['floor_res_methane_emission'] + 273*full_gdf['floor_res_n20_emission']
    full_gdf['floor_nres_co2eq_20yr'] = full_gdf['floor_nres_co2_emission'] + \
                                            82.5*full_gdf['floor_nres_methane_emission'] + 273*full_gdf['floor_nres_n20_emission']
    full_gdf['floor_res_co2eq_100yr'] = full_gdf['floor_res_co2_emission'] + \
                                            29.8*full_gdf['floor_res_methane_emission'] + 273*full_gdf['floor_res_n20_emission']
    full_gdf['floor_nres_co2eq_100yr'] = full_gdf['floor_nres_co2_emission'] + \
                                            29.8*full_gdf['floor_nres_methane_emission'] + 273*full_gdf['floor_nres_n20_emission']
    
    # get total value
    full_gdf['footprint_total_eui*area'] = full_gdf['footprint_res_eui*area'] + full_gdf['footprint_nres_eui*area']
    full_gdf['footprint_total_co2_emission'] = full_gdf['footprint_res_co2_emission'] + full_gdf['footprint_nres_co2_emission']
    full_gdf['footprint_total_methane_emission'] = full_gdf['footprint_res_methane_emission'] + full_gdf['footprint_nres_methane_emission']
    full_gdf['footprint_total_n20_emission'] = full_gdf['footprint_res_n20_emission'] + full_gdf['footprint_nres_n20_emission']
    full_gdf['floor_total_eui*area'] = full_gdf['floor_res_eui*area'] + full_gdf['floor_nres_eui*area']
    full_gdf['floor_total_co2_emission'] = full_gdf['floor_res_co2_emission'] + full_gdf['floor_nres_co2_emission']
    full_gdf['floor_total_methane_emission'] = full_gdf['floor_res_methane_emission'] + full_gdf['floor_nres_methane_emission']
    full_gdf['floor_total_n20_emission'] = full_gdf['floor_res_n20_emission'] + full_gdf['floor_nres_n20_emission']
    full_gdf['footprint_total_co2eq_20yr'] = full_gdf['footprint_res_co2eq_20yr'] + full_gdf['footprint_nres_co2eq_20yr']
    full_gdf['footprint_total_co2eq_100yr'] = full_gdf['footprint_res_co2eq_100yr'] + full_gdf['footprint_nres_co2eq_100yr']
    full_gdf['floor_total_co2eq_20yr'] = full_gdf['floor_res_co2eq_20yr'] + full_gdf['floor_nres_co2eq_20yr']
    full_gdf['floor_total_co2eq_100yr'] = full_gdf['floor_res_co2eq_100yr'] + full_gdf['floor_nres_co2eq_100yr']

    # save the full gdf chunk
    full_gdf.to_parquet(f'{calculated_full_gdf_dir}/calculated_full_gdf_chunk_{i}.parquet')
    # # transform to grid format
    # # with latitude as index and longtitude as column name, and the values are the calculated emissions
    # grid_res_eui_area  = full_gdf.pivot(index='lat', columns='lon', values='res_eui*area')
    # grid_res_eui_area.to_parquet(f'{calculated_grid_chunk_dir}/res_eui_area/grid_res_eui_area_{i}.parquet')
    # del grid_res_eui_area
    # grid_nres_eui_area = full_gdf.pivot(index='lat', columns='lon', values='nres_eui*area')
    # grid_nres_eui_area.to_parquet(f'{calculated_grid_chunk_dir}/nres_eui_area/grid_nres_eui_area_{i}.parquet')
    # del grid_nres_eui_area
    # grid_res_co2 = full_gdf.pivot(index='lat', columns='lon', values='res_co2_emission')
    # grid_res_co2.to_parquet(f'{calculated_grid_chunk_dir}/res_co2/grid_res_co2_{i}.parquet')
    # del grid_res_co2
    # grid_nres_co2  = full_gdf.pivot(index='lat', columns='lon', values='nres_co2_emission')
    # grid_nres_co2.to_parquet(f'{calculated_grid_chunk_dir}/nres_co2/grid_nres_co2_{i}.parquet')
    # del grid_nres_co2
    # grid_res_methane = full_gdf.pivot(index='lat', columns='lon', values='res_methane_emission')
    # grid_res_methane.to_parquet(f'{calculated_grid_chunk_dir}/res_methane/grid_res_methane_{i}.parquet')
    # del grid_res_methane
    # grid_nres_methane = full_gdf.pivot(index='lat', columns='lon', values='nres_methane_emission')
    # grid_nres_methane.to_parquet(f'{calculated_grid_chunk_dir}/nres_methane/grid_nres_methane_{i}.parquet')
    # del grid_nres_methane
    # grid_res_n20  = full_gdf.pivot(index='lat', columns='lon', values='res_n20_emission')
    # grid_res_n20.to_parquet(f'{calculated_grid_chunk_dir}/res_n20/grid_res_n20_{i}.parquet')
    # del grid_res_n20
    # grid_nres_n20 = full_gdf.pivot(index='lat', columns='lon', values='nres_n20_emission')
    # grid_nres_n20.to_parquet(f'{calculated_grid_chunk_dir}/nres_n20/grid_nres_n20_{i}.parquet')
    # del grid_nres_n20
    # open(calculated_grid_chunk_dir + f"{i}.txt", "w").close()
    pass

def map_gdf_grid(
    calculated_gdf_chunk_dir, dest_dir_chunked_full_gdf,calculated_full_gdf_dir, range_min=0, range_max=300
):
    """
    Multiprocessing mapping gdf to grid
    Input:
        calculated_gdf_chunk_dir: directory of calculated gdf chunks
        grid_chunk_dir: directory of grid chunks
        calculated_grid_chunk_dir: directory to save calculated grid chunks
        range_min: starting chunk number
    """
    num_processes = multiprocessing.cpu_count()  # Use the number of available CPU cores
    pool = multiprocessing.Pool(processes=num_processes)
    for i in range(range_min, range_max):
        pool.apply_async(
            process_chunk_to_grid,
            args=(
                i,
                calculated_gdf_chunk_dir,
                dest_dir_chunked_full_gdf,
                calculated_full_gdf_dir
            ),
        )# i, calculated_gdf_chunk_dir, dest_dir_chunked_full_gdf, calculated_grid_chunk_dir, calculated_full_gdf_dir
    pool.close()
    pool.join()

    pass

In [ ]:
calculated_gdf_chunk_dir = '/shared/data/building-emissions/data/estimates2020/calculated_gdf_chunks'
chunked_full_gdf = '/shared/data/building-emissions/data/estimates2020/chunked_full_gdf'
calculated_full_gdf_dir = '/shared/data/building-emissions/data/estimates2020/calculated_full_gdf_chunks'

map_gdf_grid(calculated_gdf_chunk_dir, chunked_full_gdf, calculated_full_gdf_dir, range_min=0, range_max=200)


# Concat gdf

In [ ]:
# function to concat the re-allocated emission chunk: START
def subprocess_concat(chunk_path, gdf_list):
    chunk = pd.read_parquet(chunk_path)
    gdf_list.append(chunk)


def generate_data(gdf_list):
    for item in gdf_list:
        yield item

def concat_gdf(chunk_gdf_dir, concat_gdf_path):
    print('start to concat')
    num_cores = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(
    processes=num_cores)
    manager = multiprocessing.Manager()
    gdf_list = manager.list()
    for i in range(200):
        chunk_path = os.path.join(chunk_gdf_dir, f'calculated_gdf_chunk_{i}.parquet')
        pool.apply_async(subprocess_concat, args = (chunk_path, gdf_list))
    
    pool.close()
    pool.join()
    print('processing done, start to concat')
    generator = generate_data(gdf_list)
    del gdf_list
    concat_gdf = pd.concat(generator, ignore_index=True, axis= 0)
    print('concat done, save to file')
    # save graced_grid
    concat_gdf = concat_gdf.sort_values(
        by=["lat", "lon"], ascending=[False, True]
    )
    concat_gdf.to_parquet(concat_gdf_path)
    
    print('save concat file done')
    return concat_gdf
    

# Tiff transformation

In [ ]:
import xarray as xr
def to_tif(concat_gdf_path, emission_tif_dir, emission_column_names):
    """
    Transformation to .tif file
    Input:
        graced_fine_gdf: concatenated gdf with re-allocated emissions
        re_allocated_proportinal_graced_path: path to save re-allocated GRACE emissions with proportional emissions
    Output:
        None
    """
    print("Start transformation to .tif file")
    concat_gdf = pd.read_parquet(concat_gdf_path)
    
    # concat_gdf_new = concat_gdf[
    #     ["lat", "lon", emission_column_name]
    # ].copy()

    # concat_gdf = concat_gdf_new.sort_values(
    #     by=["y", "x"], ascending=[False, True]
    # )

    latitudes = concat_gdf.y.unique()
    longitudes = concat_gdf.x.unique()
    lat_grid, lon_grid = np.meshgrid(latitudes, longitudes, indexing="ij")

    for emission_column_name in emission_column_names:  
        emission_tif_path = os.path.join(emission_tif_dir, f"{emission_column_name}.tif")
        
        graced_2d = concat_gdf[emission_column_name].values.reshape(
            lat_grid.shape
        )

        graced_xr = xr.DataArray(
            graced_2d,
            dims=["latitude", "longitude"],
            coords={"latitude": latitudes, "longitude": longitudes},
        )

        graced_xr.rio.write_crs("ESRI:54009", inplace=True)
        graced_xr.rio.to_raster(emission_tif_path)
        print("transformation to .tif file done")
    pass


In [ ]:
concat_gdf_path = '/shared/data/building-emissions/data/estimates2020/calculated_full_gdf_chunks/concat_gdf.parquet'
emission_tif_dir = '/shared/data/building-emissions/data/estimates2020/tif_files'
emission_column_names = ['footprint_res_eui*area', 'footprint_res_co2_emission', 'footprint_res_methane_emission', 'footprint_res_n20_emission', \
             'footprint_nres_eui*area', 'footprint_nres_co2_emission', 'footprint_nres_methane_emission', 'footprint_nres_n20_emission', \
            'floor_res_eui*area', 'floor_res_co2_emission', 'floor_res_methane_emission', 'floor_res_n20_emission', \
            'floor_nres_eui*area', 'floor_nres_co2_emission', 'floor_nres_methane_emission', 'floor_nres_n20_emission']
concat_gdf = concat_gdf(concat_gdf_path, emission_tif_dir, emission_column_names)

In [ ]:
# footprint
processing_dir = '/shared/data/building-emissions/data/estimates2020/tif_files'
total_emission_commands = [\
    'gdal_calc.py -A footprint_res_co2_emission.tif -B footprint_nres_co2_emission.tif --calc="A+B" --outfile=footprint_total_co2_emission.tif', \
    'gdal_calc.py -A footprint_res_methane_emission.tif -B footprint_nres_methane_emission.tif --calc="A+B" --outfile=footprint_total_methane_emission.tif', \
    'gdal_calc.py -A footprint_res_n20_emission.tif -B footprint_nres_n20_emission.tif --calc="A+B" --outfile=footprint_total_n20_emission.tif', \
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_nres_co2_emission.tif --calc="A+B" --outfile=floor_total_co2_emission.tif', \
    'gdal_calc.py -A floor_res_methane_emission.tif -B floor_nres_methane_emission.tif --calc="A+B" --outfile=floor_total_methane_emission.tif', \
    'gdal_calc.py -A floor_res_n20_emission.tif -B floor_nres_n20_emission.tif --calc="A+B" --outfile=floor_total_n20_emission.tif', \
    'gdal_calc.py -A footprint_res_co2_emission.tif -B footprint_res_methane_emission.tif -C footprint_res_n20_emission.tif  --calc="A+82.5*B+273*C" --outfile=footprint_res_co2eq_20yr.tif', \
    'gdal_calc.py -A footprint_nres_co2_emission.tif -B footprint_nres_methane_emission.tif -C footprint_nres_n20_emission.tif  --calc="A+82.5*B+273*C" --outfile=footprint_nres_co2eq_20yr.tif', \
    'gdal_calc.py -A footprint_res_co2eq_20yr.tif -B footprint_nres_co2eq_20yr.tif --calc="A+B" --outfile=footprint_total_co2eq_20yr.tif', \
    'gdal_calc.py -A footprint_res_co2_emission.tif -B footprint_res_methane_emission.tif -C footprint_res_n20_emission.tif  --calc="A+29.8*B+273*C" --outfile=footprint_res_co2eq_100yr.tif', \
    'gdal_calc.py -A footprint_nres_co2_emission.tif -B footprint_nres_methane_emission.tif -C footprint_nres_n20_emission.tif  --calc="A+29.8*B+273*C" --outfile=footprint_nres_co2eq_100yr.tif', \
    'gdal_calc.py -A footprint_res_co2eq_100yr.tif -B footprint_nres_co2eq_100yr.tif --calc="A+B" --outfile=footprint_total_co2eq_100yr.tif', \
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_res_methane_emission.tif -C floor_res_n20_emission.tif  --calc="A+82.5*B+273*C" --outfile=floor_res_co2eq_20yr.tif']
    
    
                
for command in total_emission_commands:
    subprocess.run(command, shell=True, cwd = processing_dir)

In [ ]:
# floor
processing_dir = '/shared/data/building-emissions/data/estimates2020/tif_files'
total_emission_commands = [\
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_nres_co2_emission.tif --calc="A+B" --outfile=floor_total_co2_emission.tif', \
    'gdal_calc.py -A floor_res_methane_emission.tif -B floor_nres_methane_emission.tif --calc="A+B" --outfile=floor_total_methane_emission.tif', \
    'gdal_calc.py -A floor_res_n20_emission.tif -B floor_nres_n20_emission.tif --calc="A+B" --outfile=floor_total_n20_emission.tif', \
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_nres_co2_emission.tif --calc="A+B" --outfile=floor_total_co2_emission.tif', \
    'gdal_calc.py -A floor_res_methane_emission.tif -B floor_nres_methane_emission.tif --calc="A+B" --outfile=floor_total_methane_emission.tif', \
    'gdal_calc.py -A floor_res_n20_emission.tif -B floor_nres_n20_emission.tif --calc="A+B" --outfile=floor_total_n20_emission.tif', \
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_res_methane_emission.tif -C floor_res_n20_emission.tif  --calc="A+82.5*B+273*C" --outfile=floor_res_co2eq_20yr.tif', \
    'gdal_calc.py -A floor_nres_co2_emission.tif -B floor_nres_methane_emission.tif -C floor_nres_n20_emission.tif  --calc="A+82.5*B+273*C" --outfile=floor_nres_co2eq_20yr.tif', \
    'gdal_calc.py -A floor_res_co2eq_20yr.tif -B floor_nres_co2eq_20yr.tif --calc="A+B" --outfile=floor_total_co2eq_20yr.tif', \
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_res_methane_emission.tif -C floor_res_n20_emission.tif  --calc="A+29.8*B+273*C" --outfile=floor_res_co2eq_100yr.tif', \
    'gdal_calc.py -A floor_nres_co2_emission.tif -B floor_nres_methane_emission.tif -C floor_nres_n20_emission.tif  --calc="A+29.8*B+273*C" --outfile=floor_nres_co2eq_100yr.tif', \
    'gdal_calc.py -A floor_res_co2eq_100yr.tif -B floor_nres_co2eq_100yr.tif --calc="A+B" --outfile=floor_total_co2eq_100yr.tif', \
    'gdal_calc.py -A floor_res_co2_emission.tif -B floor_res_methane_emission.tif -C floor_res_n20_emission.tif  --calc="A+82.5*B+273*C" --outfile=floor_res_co2eq_20yr.tif']
    
    
                
for command in total_emission_commands:
    subprocess.run(command, shell=True, cwd = processing_dir)